<b>Data mining Project - 2021/22</b><br/>
<span>
<b>Authors:</b> Mariagiovanna Rotundo (560765), Nunzio Lopardo (600005)</a> and Renato Eschini (203021)<br/>
<b>Group:</b>3<br/>
<b>Release date:</b> 26/12/2021
</span>

# Task 2: Data Clustering

This workbook contains a clustering analysis on the tennis dataset.
This dataset was derived from the "tennis_matches.csv" dataset by deriving player information from the matches played.

A new "performance" dataset was also derived in which the data was reprocessed to use only essential and important information for a player-related clustering analysis, eg. only players who have played more than a certain number of games will be considered, only numerical attributes, percentages strictly greater than zero, etc.

**Importing libraries**

In [ ]:
import math
from datetime import date
import re
import collections
import os

import numpy as np

import matplotlib.pyplot as plt


import scipy.stats as stats
from scipy.spatial.distance import pdist, squareform
from scipy.cluster.hierarchy import dendrogram, linkage, fcluster

from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.cluster import KMeans, DBSCAN
from sklearn.metrics import silhouette_score, pairwise_distances
from sklearn.neighbors import NearestNeighbors

import pandas as pd

from tqdm.notebook import tqdm

from pyclustering.cluster.xmeans import xmeans
from pyclustering.cluster.birch import birch
from pyclustering.cluster.center_initializer import kmeans_plusplus_initializer
from pyclustering.cluster import cluster_visualizer, cluster_visualizer_multidim
import seaborn as sns

from yellowbrick.cluster import KElbowVisualizer, SilhouetteVisualizer

**Loading the dataset**

Load "players.csv"

In [ ]:
# load of the data
DATASET_DIR = "dataset" + os.path.sep
#index_col=False say to not use the first column as ID
df_players = pd.read_csv('players.csv', sep=',', index_col=0) 
df_players.head()

Print info

In [ ]:
df_players.info()

## Perfomances dataset

We analyze the distribution of players by number of matches played

In [ ]:
sns.histplot(data=df_players['nmatch'], bins="auto", binrange=(10,400), color="lightgreen", kde=True)

In [ ]:
sns.histplot(data=df_players['best_rank'], bins="auto", binrange=(10,400), color="lightgreen", kde=True)

We create the performance dataset by taking only some attributes and the players who have at least a fixed number of n_match games (for example at least 300 games played)

In [ ]:
n_match = 100

df_performances_org = df_players[
    (df_players['best_rank']>0) & 
    (df_players['best_rank_points']>=0) & 
    (df_players['tot_minutes']>0) & 
    (df_players['ace_perc']>=0) & 
    (df_players['bpS_perc']>=0)][[
'best_rank', 
'best_rank_points',                            
'tot_minutes',
'sv1st_win', 
'sv2nd_win', 
'df', 
'ace_perc', 
'bpS_perc', 
'nmatch', 
'best_of_3_match', 
'best_of_3_wins', 
'best_of_5_match', 
'best_of_5_wins', 
'n_tourney']]
df_performances = df_performances_org.loc[df_performances_org['nmatch'] > n_match]

df_performances

## Elimination of highly correlated features

We begin by examining the correlations between the attributes of the dataset to be clustered in order to identify the highly correlated couples. Dropping redundant attributes benefits the analysis by reducing the dimensionality of the dataset and rising the influence that more useful feature could have on the whole clustering process.

With such aim in mind, we fix a maximum threshold value in order to identify highly correlated features and subsequently drop them.

In [ ]:
plt.figure(figsize = (15,6))
sns.heatmap( df_performances.corr(), annot=True)

In [ ]:
corr_threshold = 0.9
print("Att. A\t\t\tAtt. B\t\t\tCorr(A,B)")
for i in range(0, len(df_performances.columns)):
    for j in range(i+1, len(df_performances.columns)):
        corr = df_performances[df_performances.columns[i]].corr(df_performances[df_performances.columns[j]])
        if  corr > corr_threshold:
            print(df_performances.columns[i] + "\t\t\t" + df_performances.columns[j] + "\t\t\t" + '{:.4f}'.format(corr))

In [ ]:
#   
corr_columns = ['best_of_3_match', 'best_of_5_match', 'sv2nd_win', 'tot_minutes']
a = df_performances.drop(corr_columns, axis=1, inplace=False)
df_performances = a
df_performances

## K-Means

In [ ]:
# Instantiate the clustering model and visualizer
model = KMeans()
visualizer = KElbowVisualizer(model, k=(1,16))

visualizer.fit(df_performances)        # Fit the data to the visualizer
visualizer.show()        # Finalize and render the figure

In [ ]:
sse_list = list()
sil_list = list()

max_k = 15
for k in tqdm(range(2, max_k + 1), total=max_k - 1, desc="Iterating over possible K values"):
    kmeans_iter = KMeans(n_clusters=k, n_init=10, max_iter=100)
    kmeans_iter.fit(df_performances)        
    sil_list.append(silhouette_score(df_performances, kmeans_iter.labels_))
    sse = kmeans_iter.inertia_
    sse_list.append(sse)

In [ ]:
# plot indicators
fig, axs = plt.subplots(2,1,figsize=(15,25))
axs[0].plot(range(2, len(sse_list) + 2), sse_list)
axs[0].set_ylabel('SSE', fontsize=22)
axs[0].set_xlabel('K', fontsize=22)
axs[0].tick_params(axis='both', which='major', labelsize=10)

axs[1].plot(range(2, len(sil_list) + 2), sil_list)
axs[1].set_ylabel('Silhouette', fontsize=22)
axs[1].set_xlabel('K', fontsize=22)
axs[1].tick_params(axis='both', which='major', labelsize=10)

In [ ]:
n_clusters = 4

**Normalization**

In [ ]:
scaler = MinMaxScaler()
X = scaler.fit_transform(df_performances.values)

In [ ]:
model = KMeans(n_clusters)
visualizer = SilhouetteVisualizer(model)

visualizer.fit(X)        # Fit the data to the visualizer
visualizer.show()        # Finalize and render the figure

In [ ]:
kmeans = KMeans(n_clusters, n_init=10, max_iter=100)
kmeans.fit(X)

In [ ]:
np.unique(kmeans.labels_, return_counts=True)

In [ ]:
hist, bins = np.histogram(kmeans.labels_, bins=range(0, len(set(kmeans.labels_)) + 1))
dict(zip(bins, hist))

In [ ]:
kmeans.cluster_centers_

In [ ]:
# Extracting labels
cleaned_dataframe = df_performances.copy()
cleaned_dataframe["LABEL"] = kmeans.labels_

In [ ]:
# Pairplot
sns.pairplot(data = cleaned_dataframe, hue = "LABEL", palette = "Accent")

## DBscan

**Normalization**

In [ ]:
scaler = StandardScaler()
scaled_array = scaler.fit_transform(df_performances)
scaled_dataframe = pd.DataFrame( scaled_array, columns = df_performances.columns )

In [ ]:
sns.boxplot(data = scaled_dataframe, orient = "h")

In [ ]:
scaled_dataframe.describe()

In [ ]:
dbscan = DBSCAN(eps=0.75, min_samples=5)
dbscan.fit(scaled_dataframe)

In [ ]:
labels = dbscan.labels_
np.unique(dbscan.labels_, return_counts=True)

In [ ]:
cleaned_dataframe = df_performances.copy()
cleaned_dataframe["LABEL"] = labels
sns.pairplot(data = cleaned_dataframe, hue = "LABEL", palette = "Accent")

In [ ]:
dist = pdist(X=scaled_dataframe, metric='euclidean')  # pair-wise distance: how every record is far from all others
dist = squareform(dist)                      # distance matrix given the vector dist

In [ ]:
kmin, kmax = 3, 50
kth_distances = {}
for k in range(kmin, kmax + 1): # initialize k lists
    kth_distances[k] = []

In [ ]:
for d in dist:
    # argsort returns the indexes that would sort d
    index_kth_distance = np.argsort(d)[k]
    for k in range(kmin, kmax + 1):
        # append to kth_distances[k] the value in d that corresponds
        # to the distance of the i-th point (record) from its k-th nn.
        # it's like: kth_distances[k].append(sorted_d[k])), but we get "sorted_d[k]" by d[indexes_to_sort_d[k]]
        kth_distances[k].append(d[index_kth_distance])

In [ ]:
plt.figure(figsize=(50, 20))
for k in kth_distances.keys():
    plt.plot(range(0, len(kth_distances[k])), sorted(kth_distances[k]))
    
plt.ylabel('dist from k-th neighbor (eps)', fontsize=25)
plt.xlabel('sorted distances', fontsize=25)
#plt.ylim(top=5)
plt.ylim(bottom=-0.25)
plt.tick_params(axis='both', which='major', labelsize=25)
plt.grid()
plt.show()

#### Grid search for eps and min_samples

In [ ]:
def get_metrics(eps, min_samples, dataset, iter_):
    # Fitting 
    dbscan_model_ = DBSCAN(eps = eps, min_samples = min_samples)
    dbscan_model_.fit(dataset)
    
    # Mean Noise Point Distance metric
    noise_indices = dbscan_model_.labels_ == -1
    
    if True in noise_indices:
        neighboors = NearestNeighbors(n_neighbors = 6).fit(dataset)
        distances, indices = neighboors.kneighbors(dataset)
        noise_distances = distances[noise_indices, 1:]
        noise_mean_distance = round(noise_distances.mean(), 3)
    else:
        noise_mean_distance = None
    
    # Number of found Clusters metric    
    number_of_clusters = len(set(dbscan_model_.labels_[dbscan_model_.labels_ >= 0]))
    
    #print("%3d | Tested with eps = %3s and min_samples = %3s | %5s %4s" % (i, eps, min_samples, str(noise_mean_distance), number_of_clusters))
    
    return(noise_mean_distance, number_of_clusters)

In [ ]:
eps_to_test = [round(x, 3) for x in np.arange(0.4, 5, 0.3)] 
min_samples_to_test = np.arange(4, 30, 2)
eps_to_test

In [ ]:
# Dataframe per la metrica sulla distanza media dei noise points dai K punti più vicini
results_noise = pd.DataFrame( 
    data = np.zeros((len(eps_to_test),len(min_samples_to_test))), # Empty dataframe
    columns = min_samples_to_test, 
    index = eps_to_test
)

# Dataframe per la metrica sul numero di cluster
results_clusters = pd.DataFrame( 
    data = np.zeros((len(eps_to_test),len(min_samples_to_test))), # Empty dataframe
    columns = min_samples_to_test, 
    index = eps_to_test
)

In [ ]:
#grid search

In [ ]:
i = 0

for eps in eps_to_test:
    for min_samples in min_samples_to_test:
        i += 1
        # Calcolo le metriche
        noise_metric, cluster_metric  = get_metrics(eps, min_samples, scaled_dataframe, i)
        # Inserisco i risultati nei relativi dataframe
        results_noise.loc[eps, min_samples] = noise_metric
        results_clusters.loc[eps, min_samples] = cluster_metric

In [ ]:
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16,8) )

sns.heatmap(results_noise, annot = True, ax = ax1, cbar = False).set_title("Mean Noise Points Distance")
sns.heatmap(results_clusters, annot = True, ax = ax2, cbar = False).set_title("Number of clusters")

ax1.set_xlabel("min_samples")
ax2.set_xlabel("min_samples")
ax1.set_ylabel("eps")
ax2.set_ylabel("eps")

plt.tight_layout()
plt.show()

#### choose of parameters 

In [ ]:
best_dbscan = DBSCAN(eps = 1, min_samples = 14)
# Fitting
best_dbscan.fit(scaled_dataframe)

labels = best_dbscan.labels_
np.unique(best_dbscan.labels_, return_counts=True)

In [ ]:
#scaled_dataframe["LABEL"] = best_dbscan.labels_
#sns.pairplot(data = scaled_dataframe, hue = "LABEL")

# Extracting labels
cleaned_dataframe = df_performances.copy()
cleaned_dataframe["LABEL"] = best_dbscan.labels_
# Pairplot
sns.pairplot(data = cleaned_dataframe, hue = "LABEL", palette = "Accent")

**Normalization**

In [ ]:
scaler = MinMaxScaler()
scaled_array = scaler.fit_transform(df_performances)
scaled_dataframe = pd.DataFrame( scaled_array, columns = df_performances.columns )

In [ ]:
dbscan = DBSCAN(eps=0.75, min_samples=5)
dbscan.fit(scaled_dataframe)
labels = dbscan.labels_
np.unique(dbscan.labels_, return_counts=True)

In [ ]:
cleaned_dataframe = df_performances.copy()
cleaned_dataframe["LABEL"] = labels
sns.pairplot(data = cleaned_dataframe, hue = "LABEL", palette = "Accent")

In [ ]:
dist = pdist(X=scaled_dataframe, metric='euclidean')  # pair-wise distance: how every record is far from all others
dist = squareform(dist)                      # distance matrix given the vector dist

In [ ]:
kmin, kmax = 3, 50
kth_distances = {}
for k in range(kmin, kmax + 1): # initialize k lists
    kth_distances[k] = []

In [ ]:
for d in dist:
    # argsort returns the indexes that would sort d
    index_kth_distance = np.argsort(d)[k]
    for k in range(kmin, kmax + 1):
        # append to kth_distances[k] the value in d that corresponds
        # to the distance of the i-th point (record) from its k-th nn.
        # it's like: kth_distances[k].append(sorted_d[k])), but we get "sorted_d[k]" by d[indexes_to_sort_d[k]]
        kth_distances[k].append(d[index_kth_distance])

In [ ]:
plt.figure(figsize=(50, 20))
for k in kth_distances.keys():
    plt.plot(range(0, len(kth_distances[k])), sorted(kth_distances[k]))
    
plt.ylabel('dist from k-th neighbor (eps)', fontsize=25)
plt.xlabel('sorted distances', fontsize=25)
#plt.ylim(top=5)
plt.ylim(bottom=-0.25)
plt.tick_params(axis='both', which='major', labelsize=25)
plt.grid()
plt.show()

In [ ]:
eps_to_test = [round(x, 3) for x in np.arange(0.2, 0.6, 0.1)]
min_samples_to_test = np.arange(2, 30, 2)
eps_to_test

In [ ]:
# Dataframe per la metrica sulla distanza media dei noise points dai K punti più vicini
results_noise = pd.DataFrame( 
    data = np.zeros((len(eps_to_test),len(min_samples_to_test))), # Empty dataframe
    columns = min_samples_to_test, 
    index = eps_to_test
)

# Dataframe per la metrica sul numero di cluster
results_clusters = pd.DataFrame( 
    data = np.zeros((len(eps_to_test),len(min_samples_to_test))), # Empty dataframe
    columns = min_samples_to_test, 
    index = eps_to_test
)

In [ ]:
i = 0

for eps in eps_to_test:
    for min_samples in min_samples_to_test:
        i += 1
        # Calcolo le metriche
        noise_metric, cluster_metric  = get_metrics(eps, min_samples, scaled_dataframe, i)
        # Inserisco i risultati nei relativi dataframe
        results_noise.loc[eps, min_samples] = noise_metric
        results_clusters.loc[eps, min_samples] = cluster_metric

In [ ]:
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16,4) )

sns.heatmap(results_noise, annot = True, ax = ax1, cbar = False).set_title("Mean Noise Points Distance")
sns.heatmap(results_clusters, annot = True, ax = ax2, cbar = False).set_title("Number of clusters")

ax1.set_xlabel("min_samples")
ax2.set_xlabel("min_samples")
ax1.set_ylabel("eps")
ax2.set_ylabel("eps")

plt.tight_layout()
plt.show()

In [ ]:
best_dbscan = DBSCAN(eps = 0.2, min_samples = 28)
# Fitting
best_dbscan.fit(scaled_dataframe)

labels = best_dbscan.labels_
np.unique(best_dbscan.labels_, return_counts=True)

In [ ]:
# Extracting labels
cleaned_dataframe = df_performances.copy()
cleaned_dataframe["LABEL"] = best_dbscan.labels_
# Pairplot
sns.pairplot(data = cleaned_dataframe, hue = "LABEL", palette = "Accent")

# Hierachical

In this section we will see hierarchical clustering performed the divisive technique, in particular, using the four methods to compute the distances between clusters. To compute the distances has been used euclidean distance.

In [ ]:
from scipy.cluster.hierarchy import dendrogram, linkage, fcluster

In [ ]:
def count_cluster_elements(data, threshold, criterion='distance'):
    count = {}
    clusters = fcluster(data, threshold, criterion)
    for c in clusters:
        count[c] = count[c]+1 if c in count else 1
    return count, clusters

In [ ]:
methods = ['single', 'complete', 'average', 'ward']
metrics = ['silhouette', 'davies_bouldin']
evaluation_metrics = pd.DataFrame(index=methods, columns=metrics)
models = {}

Using scipy we can compute the linkage matrix and use it for the dendrogram plot. This is done for all the criteria.

- ‘single’ uses the minimum of the distances between all observations of the two sets.

- ‘complete’ or ‘maximum’ linkage uses the maximum distances between all observations of the two sets.

- ‘average’ uses the average of the distances of each observation of the two sets.

- ‘ward’ minimizes the variance of the clusters being merged.

In [ ]:
scaler = MinMaxScaler()
scaled_array = scaler.fit_transform(df_performances)
scaled_dataframe = pd.DataFrame( scaled_array, columns = df_performances.columns )

In [ ]:
for method in methods:
    link = linkage(scaled_dataframe, method=method, metric = 'euclidean')
    models[method] = link

Now plot the four dendrograms.

In [ ]:
f, axs = plt.subplots(ncols=4, figsize=(32,7))
i = 0
for model in models.keys():
    axs[i].set_title('Hierarchical Clustering by ' + model + ' Algorithm (' + methods[i] + '-linkage)')
    axs[i].set_xlabel('Players or (Cluster Size)')
    axs[i].set_ylabel('Distance')
    dend = dendrogram(models[model],ax=axs[i],truncate_mode='lastp', p=25, leaf_rotation=60, leaf_font_size = 8, show_contracted=True)
    i+=1

DA FINIRE -- Looking at the previews dendrogram 

In [ ]:
cut_threshold = {methods[0]:0.48, methods[1]:1.8,methods[2]:1,methods[3]:10}

In [ ]:
f, axs = plt.subplots(ncols=4, figsize=(32,7))
i = 0
for model in models.keys():
    axs[i].set_title('Hierarchical Clustering by ' + model + ' Algorithm (' + methods[i] + '-linkage)')
    axs[i].set_xlabel('Players or (Cluster Size)')
    axs[i].set_ylabel('Distance')
    axs[i].axhline(y=cut_threshold[model], color="black")
    dend = dendrogram(models[model],ax=axs[i],truncate_mode='lastp', p=25, leaf_rotation=60, leaf_font_size = 8, show_contracted=True)
    i+=1


**Cluster evalution**

Let's compute some metrics to evaluate the clustering quality.

In [ ]:
from sklearn.cluster import AgglomerativeClustering
from sklearn.metrics import davies_bouldin_score, silhouette_score
model_labels = {}
for model in models.keys():
    print(model)
    count, clusters = count_cluster_elements(models[model], cut_threshold[model])
    print(count)
    
    cluster = AgglomerativeClustering(n_clusters=len(count.keys()), affinity='euclidean', linkage=model)
    cluster.fit_predict(scaled_dataframe)
    labels = cluster.labels_
    model_labels[model] = labels
    davies_bouldin = davies_bouldin_score(scaled_dataframe, labels)
    silhouette = silhouette_score(scaled_dataframe, labels, metric='euclidean')
    evaluation_metrics.loc[model][metrics[0]] = silhouette
    evaluation_metrics.loc[model][metrics[1]] = davies_bouldin
evaluation_metrics

In [ ]:
i = 0
f, axs = plt.subplots(ncols=4, figsize=(32,7), sharey=True)
for model in models.keys():
    axs[i].set_title('Hierarchical Clustering by ' + model + ' Algorithm (' + methods[i] + '-linkage)')
    axs[i].scatter(df_performances['nmatch'].values, df_performances['best_rank_points'].values, c=model_labels[model] , s=25, cmap='viridis')
    axs[i].set_xlabel('nmatch')
    axs[i].set_ylabel('best_rank_points')
    i+=1

# XMEANS

In [ ]:
amount_initial_centers = 1   #number of clusters at the start. xmeans starts with only one cluster
max_n_clusters = 20

initial_centers = kmeans_plusplus_initializer(df_performances, amount_initial_centers).initialize()
xmeans_instance = xmeans(df_performances, initial_centers, kmax=max_n_clusters)
xmeans_instance.process(); #split with bayesian Information Criterion

clusters = xmeans_instance.get_clusters();
centers = xmeans_instance.get_centers()

print([len(c) for c in clusters])   

In [ ]:
# display allocated clusters
visualizer = cluster_visualizer_multidim();
visualizer.append_clusters(clusters, df_performances.values.tolist())
visualizer.append_cluster(centers, None, marker = '*', markersize=5)
#visualizer.show()
visualizer.show(pair_filter=[[0, 1], [0, 2], [0, 3]])

**plot with sns library**

In [ ]:
labels = np.zeros(df_performances.shape[0],  dtype=int) #num of rows
for i in range(len(clusters)):#number of cluster
    for j in clusters[i]: #index of row of dataset in cluster i
        labels[j] = int(i)

In [ ]:
palette_n = sns.color_palette("hls", n_colors=len(clusters))
palette_n

In [ ]:
# Extracting labels
cleaned_dataframe = df_performances.copy()
cleaned_dataframe["LABEL"] = labels
# Pairplot
sns.pairplot(data = cleaned_dataframe, hue = "LABEL", palette = palette_n)

# BIRCH (Balanced Iterative Reducing and Clustering using Hierarchies) 

<a href="https://pyclustering.github.io/docs/0.10.1/html/dc/d83/namespacepyclustering_1_1cluster_1_1birch.html">BIRCH by PYCLUSTERING</a>
<br />
<a href="https://pyclustering.github.io/docs/0.10.1/html/d0/de3/citelist.html#CITEREF_article::birch::1">BIRCH PAPER</a>

In [ ]:
data = df_performances.values.tolist()
 
# Create BIRCH algorithm
birch_instance = birch(data, 3, diameter=3)
 
# Cluster analysis
birch_instance.process()
 
# Obtain results of clustering
clusters = birch_instance.get_clusters()

print([len(c) for c in clusters]) 

In [ ]:
# visualize obtained clusters in multi-dimensional space
visualizer = cluster_visualizer_multidim()
visualizer.append_clusters(clusters, data = df_performances.values.tolist())
visualizer.show(pair_filter=[[0, 1], [0, 2], [0, 3]])

In [ ]:
labels = np.zeros(df_performances.shape[0],  dtype=int) #num of rows
for i in range(len(clusters)):#number of cluster
    for j in clusters[i]: #index of row of dataset in cluster i
        labels[j] = int(i)

In [ ]:
palette_n = sns.color_palette("hls", n_colors=len(clusters))
palette_n

In [ ]:
# Extracting labels
cleaned_dataframe = df_performances.copy()
cleaned_dataframe["LABEL"] = labels
# Pairplot
sns.pairplot(data = cleaned_dataframe, hue = "LABEL", palette = palette_n)